In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import itertools
import math
from collections import defaultdict

import networkx as nx
from networkx.utils import py_random_state

from networkx.generators.classic import complete_graph, empty_graph, path_graph, star_graph
from networkx.generators.degree_seq import degree_sequence_tree

In [ ]:
def DAG_ER_generator(num_graphs, num_nodes, edge_prob):
    if not 0 <= edge_prob <= 1:
        raise ValueError('Improper probability')
    if num_graphs < 1 or num_nodes < 1:
        raise ValueError('Graphs and nodes must be greater than 1')
    random_matrices = np.random.random((int(num_graphs), int(num_nodes), int(num_nodes)))
    upper_tri_mask = np.triu(np.ones((num_nodes, num_nodes), dtype=bool), k=1)
    G = random_matrices < edge_prob
    G = np.logical_and(G, upper_tri_mask[np.newaxis, :, :])
    return G
def DAG_DBA_generator(num_graphs, num_nodes, num_p_edges, num_p_minus_one_edges, edge_prob):
    if not 0 < edge_prob < 1:
        raise ValueError('Improper probability')
    if num_graphs < 1 or num_nodes < 1:
        raise ValueError('Graphs and nodes must be greater than 1')
    size_of_first_star = max(num_p_edges, num_p_minus_one_edges)
    if size_of_first_star > num_nodes - 1:
        raise ValueError('num_p_edges and num_p_minus_one_edges must be less than num_nodes')

    graphs = np.zeros(shape=(num_graphs, num_nodes, num_nodes), dtype=bool)
    
    for idx in range(num_graphs):
        G = np.zeros((num_nodes, num_nodes), dtype=bool)
        # Initial active node(s) with out-edges
        G[0, 1:size_of_first_star] = True  # Assuming nodes 0 to size_of_first_star-1 can initially point to node 0
        
        for target in range(size_of_first_star, num_nodes):
            potential_sources = np.arange(target)
            out_degrees = G[:target, :].sum(axis=1)  # Sum over rows to get out-degrees
            probs = (out_degrees + 1) / (out_degrees + 1).sum()
            m = num_p_edges if np.random.random() < edge_prob else num_p_minus_one_edges
            sources = np.random.choice(potential_sources, size=m, replace=False, p=probs)
            G[sources, target] = True

        graphs[idx] = G
    
    return graphs
            
def rich_get_richer_DAG(num_graphs, num_nodes, num_star_edges, num_starting_stars, edge_prob):
    if not 0 < edge_prob < 1:
        raise ValueError('Improper probability')
    if num_graphs < 1 or num_nodes < 1:
        raise ValueError('Graphs and nodes must be greater than 1')
    if num_starting_stars + num_starting_stars * star_size > num_nodes - 1:
        raise ValueError(' num_starting_stars + num_starting_stars * num_star_edges must be less than num_nodes')

    graphs = np.zeros(shape=(num_graphs, num_nodes, num_nodes), dtype=bool)
    
    for idx in range(num_graphs):
        G = np.zeros((num_nodes, num_nodes), dtype=bool)
        # Initial active node(s) with out-edges
        for x in range(num_starting_stars):
            G[x, x*star_size+1:(x+1)*star_size] = True
        
        for target in range(num_starting_stars*star_size, num_nodes):
            potential_sources = np.arange(target)
            out_degrees = G[:target, :].sum(axis=1)  # Sum over rows to get out-degrees
            probs = (out_degrees + 1) / (out_degrees + 1).sum()
            m = num_p_edges if np.random.random() < edge_prob else num_p_minus_one_edges
            sources = np.random.choice(potential_sources, size=m, replace=False, p=probs)
            G[sources, target] = True

        graphs[idx] = G
    
    return graphs

In [ ]:
def make_acyclic(G):
    """
    Removes edges to make a graph G acyclic.
    """
    while not is_directed_acyclic_graph(G):
        try:
            # Identify a cycle
            cycle = nx.find_cycle(G, orientation='original')
            # Remove an edge from the cycle
            G.remove_edge(cycle[0][0], cycle[0][1])
        except nx.NetworkXNoCycle:
            break  # Break the loop if no cycle is found
    return G
def generate_ER_graph_dataset(n = 16, total = 1000):
    graph_dataset = []
    for x in range(total):
            G_er = nx.fast_gnp_random_graph(n, .3, directed=True)
            
            G_er = make_acyclic(G_er)
            G_er = nx.adjacency_matrix(G_er).todense()
            graph_dataset.append(G_er)
    return graph_dataset
def generate_Watts(n = 16, total = 1000):
    graph_dataset = []
    for x in range(total):
        G_ba = nx.barabasi_albert_graph(n, 2)
        G_ba = nx.DiGraph(G_ba)  # Convert to directed graph
        G_ba = make_acyclic(G_ba)
        G_ba = nx.adjacency_matrix(G_ba).todense()
        graph_dataset.append(G_ba)
    return graph_dataset
def generate_diverse_graph_dataset(n = 16, total = 1000):
    graph_dataset = []
    for x in range(total):
        which_graph = np.random.randint(0, 4)
        if which_graph == 0:
            G_er = nx.gnm_random_graph(n, int(n*1.5), directed=True)
            G_er = make_acyclic(G_er)
            G_er = nx.adjacency_matrix(G_er).todense()
            graph_dataset.append(G_er)
        # Watts-Strogatz (need to modify for DAG)
        elif which_graph == 1:
            G_ws = nx.watts_strogatz_graph(n, 4, 0.5)
            G_ws = nx.DiGraph(G_ws)  # Convert to directed graph
            G_ws = make_acyclic(G_ws)
            G_ws = nx.adjacency_matrix(G_ws).todense()
            graph_dataset.append(G_ws)
        # Barabasi-Albert (need to modify for DAG)
        elif which_graph == 2:
            G_ba = nx.barabasi_albert_graph(n, 2)
            G_ba = nx.DiGraph(G_ba)  # Convert to directed graph
            G_ba = make_acyclic(G_ba)
            G_ba = nx.adjacency_matrix(G_ba).todense()
            graph_dataset.append(G_ba)
        # Stochastic Block Model
        else:
            # Ensure the number of blocks is a power of 2 and divides 'n' evenly
            num_blocks = 2**np.random.randint(1, int(math.log2(n)) + 1)
            block_size = n // num_blocks
            sizes = [block_size] * num_blocks  # Equal-sized blocks
            
            # Generating a random p_matrix
            p_matrix = np.random.rand(num_blocks, num_blocks) / 20
            
            G_sbm = nx.stochastic_block_model(sizes, p_matrix.tolist(), directed=True)
            G_sbm = make_acyclic(G_sbm)
            G_sbm = nx.to_numpy_array(G_sbm)
            graph_dataset.append(G_sbm)
    return graph_dataset